In [162]:
from pathlib import Path

import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
np.set_printoptions(precision=4)

## SketchRNN

In [90]:

DOWNLOAD_ROOT = "http://download.tensorflow.org/data/"
FILENAME = "quickdraw_tutorial_dataset_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME,
                                DOWNLOAD_ROOT + FILENAME,
                                cache_subdir="datasets/quickdraw",
                                extract=True)

1065301781/1065301781 [==============================] - 132s 0us/step


In [6]:
filepath

'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets/quickdraw\\quickdraw_tutorial_dataset_v1.tar.gz'

In [72]:
quickdraw_dir = Path(filepath).parent
all_files= sorted([str(file) for file in quickdraw_dir.glob("*.*")])
train_files = sorted([str(path) for path in quickdraw_dir.glob("training.tfrecord-*")])
eval_files = sorted([str(path) for path in quickdraw_dir.glob("eval.tfrecord-*")])

In [73]:
all_files

['C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00000-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00001-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00002-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00003-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00004-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00005-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00006-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00007-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00008-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00009-of-00010',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord.classes',
 'C:\\Users\\sec_zca9cu_rw\\.keras\\dat

In [15]:
#读class 文件
with open(quickdraw_dir / "eval.tfrecord.classes") as test_classes_file:
    test_classes = test_classes_file.readlines()
    
with open(quickdraw_dir / "training.tfrecord.classes") as train_classes_file:
    train_classes = train_classes_file.readlines()

In [81]:
assert test_classes==train_classes #验证测试类别标签和训练标签类别一致
class_names = [name.strip().lower() for name in train_classes] #去除类别标签前后空格，转小写
len(class_names) #345个分类

345

In [42]:
def parse(data_batch):
    feature_descriptions = {
        "ink": tf.io.VarLenFeature(dtype=tf.float32),#变长特征
        "shape": tf.io.FixedLenFeature([2], dtype=tf.int64),#定长特征
        "class_index": tf.io.FixedLenFeature([1], dtype=tf.int64)
    }
    examples = tf.io.parse_example(data_batch, feature_descriptions)
    flat_sketches = tf.sparse.to_dense(examples["ink"])
    sketches = tf.reshape(flat_sketches, shape=[tf.size(data_batch), -1, 3])
    lengths = examples["shape"][:, 0]
    labels = examples["class_index"][:, 0]
    return sketches, lengths, labels

In [43]:
def quickdraw_dataset(filepaths, batch_size=32, shuffle_buffer_size=None,
                      n_parse_threads=5, n_read_threads=5, cache=False):
    dataset = tf.data.TFRecordDataset(filepaths,
                                      num_parallel_reads=n_read_threads)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(parse, num_parallel_calls=n_parse_threads)
    return dataset.prefetch(1)

In [74]:
examplefile='C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\quickdraw\\eval.tfrecord-00004-of-00010'
dataset = tf.data.TFRecordDataset(examplefile)
bytedataset=list(dataset)

In [75]:
feature_descriptions = {
        "ink": tf.io.VarLenFeature(dtype=tf.float32),#变长特征
        "shape": tf.io.FixedLenFeature([2], dtype=tf.int64),#定长特征
        "class_index": tf.io.FixedLenFeature([1], dtype=tf.int64)
    }
dataset_example = tf.io.parse_example(bytedataset, feature_descriptions)

In [78]:
tf.sparse.to_dense(dataset_example["ink"])

<tf.Tensor: shape=(34279, 2040), dtype=float32, numpy=
array([[-0.0863, -0.0354,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [-0.0472, -0.019 ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [-0.0748, -0.0442,  0.    , ...,  0.    ,  0.    ,  0.    ],
       ...,
       [ 0.1391,  0.0157,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.0235, -0.0714,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.0902, -0.0127,  0.    , ...,  0.    ,  0.    ,  0.    ]],
      dtype=float32)>

## jsb_chorales

In [163]:
chorales_file = tf.keras.utils.get_file('jsb_chorales', 
                                       'https://raw.githubusercontent.com/ageron/data/main/jsb_chorales.tgz',
                                        untar=True)

chorales_root = Path(chorales_file)

train_chorales_list=sorted([str(file) for file in chorales_root.glob("train/*.csv")])
valid_chorales_list=sorted([str(file) for file in chorales_root.glob("valid/*.csv")])
test_chorales_list=sorted([str(file) for file in chorales_root.glob("test/*.csv")])


In [43]:
#看下基本数据结构，四个int64列
csv_example='C:\\Users\\sec_zca9cu_rw\\.keras\\datasets\\jsb_chorales\\test\\chorale_305.csv'

df = pd.read_csv(csv_example)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   note0   228 non-null    int64
 1   note1   228 non-null    int64
 2   note2   228 non-null    int64
 3   note3   228 non-null    int64
dtypes: int64(4)
memory usage: 7.2 KB


In [171]:
import pandas as pd

def load_chorales(filepaths):
    return [pd.read_csv(filepath).values.tolist() for filepath in filepaths]

train_chorales = load_chorales(train_chorales_list)
valid_chorales = load_chorales(valid_chorales_list)
test_chorales = load_chorales(test_chorales_list)

In [6]:

notes = set() #空集
for chorales in (train_chorales, valid_chorales, test_chorales):
    for chorale in chorales:
        for chord in chorale:
            notes |= set(chord) #并集

n_notes = len(notes)
min_note = min(notes - {0}) #去除0元素，即休止符
max_note = max(notes)

assert min_note == 36 #C1 = C on octave 1 第一个八度的C
assert max_note == 81 #A5 = A on octave 5 第五个八度的A

In [7]:
from IPython.display import Audio

def notes_to_frequencies(notes):
    # Frequency doubles when you go up one octave; there are 12 semi-tones
    # per octave; Note A on octave 4 is 440 Hz, and it is note number 69.
    return 2 ** ((np.array(notes) - 69) / 12) * 440

def frequencies_to_samples(frequencies, tempo, sample_rate):
    note_duration = 60 / tempo # the tempo is measured in beats per minutes 一个音持续多少秒
    # To reduce click sound at every beat, we round the frequencies to try to
    # get the samples close to zero at the end of each note.
    frequencies = np.round(note_duration * frequencies) / note_duration #对频率*时长取整
    n_samples = int(note_duration * sample_rate) #一个音符需要采样多少个音频信号
    time = np.linspace(0, note_duration, n_samples)#一个音对采样频率等分时间
    sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time) #frequencies.reshape(-1, 1)为array([[frequencies]])
    # Removing all notes with frequencies ≤ 9 Hz (includes note 0 = silence)
    sine_waves *= (frequencies > 9.).reshape(-1, 1)
    return sine_waves.reshape(-1)#其实不太能get这个reshape到(-1,1)乘布尔值再reshape(-1)的操作

def chords_to_samples(chords, tempo, sample_rate):
    freqs = notes_to_frequencies(chords)
    freqs = np.r_[freqs, freqs[-1:]] # make last note a bit longer 把频率列表最后一个值重复一遍
    merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                     for melody in freqs.T], axis=0) #把每个声部的频率转换为信号，再取平均，等同于傅里叶级数
    n_fade_out_samples = sample_rate * 60 // tempo # fade out last note 向下取整除法，获取最后一个音的采样数
    fade_out = np.linspace(1., 0., n_fade_out_samples)**2 
    merged[-n_fade_out_samples:] *= fade_out
    return merged

def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
    samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
    if filepath:
        from scipy.io import wavfile
        samples = (2**15 * samples).astype(np.int16)
        wavfile.write(filepath, sample_rate, samples)
        return display(Audio(filepath))
    else:
        return display(Audio(samples, rate=sample_rate))

In [108]:
tempo=160
sample_rate=44100

freqs=notes_to_frequencies(train_chorales[np.random.randint(0,len(train_chorales_list))]) #随机取一个文件，音高转换为频率
freqclip=freqs[0:np.random.randint(0,len(freqs))] #随机取里面一段片段

merged= np.mean([frequencies_to_samples(melody, tempo=tempo, sample_rate=sample_rate) for melody in freqclip.T],axis=0)

display(Audio(merged, rate=sample_rate)) 

### dataset序列，batch操作

In [143]:
range_ds = tf.data.Dataset.range(1000)
batches = range_ds.batch(10, drop_remainder=True)
#whether the last batch should be dropped in the case it has fewer than `batch_size` elements
#the default behavior is not to drop the smaller batch

for batch in batches.take(5):
    print(batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]


In [113]:
#对未来进行一步密集预测，将特征和标签相对彼此移动一步
def dense_1_step(batch):
  # Shift features and labels one step relative to each other.
  return batch[:-1], batch[1:] #feature 为该batch除去最后一个的剩余序列，label为该batch出去第一个的剩余序列

predict_dense_1_step = batches.map(dense_1_step)

for features, label in predict_dense_1_step.take(3):
  print(features.numpy(), " => ", label.numpy())

[0 1 2 3 4 5 6 7 8]  =>  [1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18]  =>  [11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28]  =>  [21 22 23 24 25 26 27 28 29]


In [117]:
#预测整个窗口而非一个固定偏移量
batches = range_ds.batch(15, drop_remainder=True) #15个数据一批数据

def label_next_5_steps(batch):
    return batch[:-5], batch[-5:]  #Inputs: All except the last 5 steps Labels: The last 5 steps

predict_5_steps = batches.map(label_next_5_steps)

for features, label in predict_5_steps.take(3):
    print(features.numpy(), " => ", label.numpy())

[0 1 2 3 4 5 6 7 8 9]  =>  [10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]  =>  [25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]  =>  [40 41 42 43 44]


In [118]:
#一个批次的特征和另一批次的标签部分重叠，使用dataset.zip 函数
feature_length = 10
label_length = 3

features = range_ds.batch(feature_length, drop_remainder=True)
labels = range_ds.batch(feature_length).skip(1).map(lambda labels: labels[:label_length]) #取下一批的前三个作为当前一批的预测值

predicted_steps = tf.data.Dataset.zip((features, labels))

for features, label in predicted_steps.take(5):
  print(features.numpy(), " => ", label.numpy())

[0 1 2 3 4 5 6 7 8 9]  =>  [10 11 12]
[10 11 12 13 14 15 16 17 18 19]  =>  [20 21 22]
[20 21 22 23 24 25 26 27 28 29]  =>  [30 31 32]
[30 31 32 33 34 35 36 37 38 39]  =>  [40 41 42]
[40 41 42 43 44 45 46 47 48 49]  =>  [50 51 52]


### dataset序列，window操作

In [159]:
window_size = 5
#window是由 Datasets 组成的 Dataset
windows = range_ds.window(window_size, shift=1) #窗口为5，步进为1
for sub_ds in windows.take(5):
    print(sub_ds)

#Dataset.flat_map 方法获取由数据集组成的数据集，并将其合并为一个数据集    
for x in windows.flat_map(lambda x: x).take(30):
   print(x.numpy(),end=' ')

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
0 1 2 3 4 1 2 3 4 5 2 3 4 5 6 3 4 5 6 7 4 5 6 7 8 5 6 7 8 9 

In [133]:
#结合batch函数

def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

for example in windows.flat_map(sub_to_batch).take(5):
    print(example.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]


In [156]:
#结合shift stride 参数
def make_window_dataset(ds, window_size, shift, stride):
    windows = ds.window(window_size, shift=shift, stride=stride)

    def sub_to_batch(sub):
        return sub.batch(window_size, drop_remainder=True)

    windows = windows.flat_map(sub_to_batch)
    return windows

ds = make_window_dataset(range_ds, window_size=10, shift=5, stride=3)

for example in ds.take(10):
    print(example.numpy())

[ 0  3  6  9 12 15 18 21 24 27]
[ 5  8 11 14 17 20 23 26 29 32]
[10 13 16 19 22 25 28 31 34 37]
[15 18 21 24 27 30 33 36 39 42]
[20 23 26 29 32 35 38 41 44 47]
[25 28 31 34 37 40 43 46 49 52]
[30 33 36 39 42 45 48 51 54 57]
[35 38 41 44 47 50 53 56 59 62]
[40 43 46 49 52 55 58 61 64 67]
[45 48 51 54 57 60 63 66 69 72]


In order to be able to generate new chorales, we want to train a model that can predict the next chord given all the previous chords. If we naively try to predict the next chord in one shot, predicting all 4 notes at once, we run the risk of getting notes that don't go very well together (believe me, I tried). It's much better and simpler to predict one note at a time. So we will need to preprocess every chorale, turning each chord into an arpegio (i.e., a sequence of notes rather than notes played simultaneuously). So each chorale will be a long sequence of notes (rather than chords), and we can just train a model that can predict the next note given all the previous notes. We will use a sequence-to-sequence approach, where we feed a window to the neural net, and it tries to predict that same window shifted one time step into the future.

We will also shift the values so that they range from 0 to 46, where 0 represents silence, and values 1 to 46 represent notes 36 (C1) to 81 (A5).

And we will train the model on windows of 128 notes (i.e., 32 chords).

Since the dataset fits in memory, we could preprocess the chorales in RAM using any Python code we like, but I will demonstrate here how to do all the preprocessing using tf.data (there will be more details about creating windows using tf.data in the next chapter).

In [254]:
def create_target(batch):
    X = batch[:, :-1]
    Y = batch[:, 1:] # predict next note in each arpegio, at each step ,arpegio即音符序列
    return X, Y

def preprocess(window):
    #tf.where(input_tensor, a,b)
    #对于张量a，如果input_tensor对应位置的元素为True，则张量a中的该位置处元素保留，反之由张量b中相应位置的元素来代替
    # shift values 1 to 46 represent notes 36 (C1) to 81 (A5)
    window = tf.where(window == 0, window, window - min_note + 1)
    return tf.reshape(window, [-1]) # convert to arpegio

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale): #对可变张量的每一个元素（即一首赞美诗），窗口化处理
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        return dataset.flat_map(batch_window)

    chorales = tf.ragged.constant(chorales, ragged_rank=1)#可变长度，不规则张量
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(to_windows).map(preprocess) #对于每个不规则张量（一首赞美诗）
    #flap_map操作即标志着窗口化后不再处理曲子间隔，以window_size+1作为单位进行处理
    #preprocess处理完后一个时间序列为33*4=132个音符，步长16
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size) #batch处理后，32个时间序列为一批
    dataset = dataset.map(create_target)#map后，X为每一个序列的[:-1],Y为每个序列的[1:]
    return dataset.prefetch(1)

In [289]:
train_set = bach_dataset(train_chorales, shuffle_buffer_size=1000)
valid_set = bach_dataset(valid_chorales)
test_set = bach_dataset(test_chorales)

We could feed the note values directly to the model, as floats, but this would probably not give good results. Indeed, the relationships between notes are not that simple: for example, if you replace a C3 with a C4, the melody will still sound fine, even though these notes are 12 semi-tones apart (i.e., one octave). Conversely, if you replace a C3 with a C#3, it's very likely that the chord will sound horrible, despite these notes being just next to each other. So we will use an Embedding layer to convert each note to a small vector representation (see Chapter 16 for more details on embeddings). We will use 5-dimensional embeddings, so the output of this first layer will have a shape of [batch_size, window_size, 5].

We will then feed this data to a small WaveNet-like neural network, composed of a stack of 4 Conv1D layers with doubling dilation rates. We will intersperse these layers with BatchNormalization layers for faster better convergence.
Then one LSTM layer to try to capture long-term patterns.

And finally a Dense layer to produce the final note probabilities. It will predict one probability for each chorale in the batch, for each time step, and for each possible note (including silence). So the output shape will be [batch_size, window_size, 47].

In [290]:
n_embedding_dims = 5

model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=n_notes, output_dim=n_embedding_dims,
                           input_shape=[None]),
    keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.Dense(n_notes, activation="softmax")#note probabilities
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 5)           235       
                                                                 
 conv1d (Conv1D)             (None, None, 32)          352       
                                                                 
 batch_normalization (Batch  (None, None, 32)          128       
 Normalization)                                                  
                                                                 
 conv1d_1 (Conv1D)           (None, None, 48)          3120      
                                                                 
 batch_normalization_1 (Bat  (None, None, 48)          192       
 chNormalization)                                                
                                                                 
 conv1d_2 (Conv1D)           (None, None, 64)          6

In [293]:
optimizer = keras.optimizers.Nadam(learning_rate=1e-3)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

callback=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
history=model.fit(train_set, epochs=30, validation_data=valid_set,callbacks=callback)

Epoch 1/30
98/98 [==============================] - 35s 279ms/step - loss: 1.8964 - accuracy: 0.5209 - val_loss: 3.5425 - val_accuracy: 0.0797
Epoch 2/30
98/98 [==============================] - 27s 271ms/step - loss: 0.8855 - accuracy: 0.7662 - val_loss: 3.4698 - val_accuracy: 0.0718
Epoch 3/30
98/98 [==============================] - 26s 267ms/step - loss: 0.7368 - accuracy: 0.7944 - val_loss: 2.9692 - val_accuracy: 0.2212
Epoch 4/30
98/98 [==============================] - 26s 269ms/step - loss: 0.6621 - accuracy: 0.8099 - val_loss: 2.2466 - val_accuracy: 0.3652
Epoch 5/30
98/98 [==============================] - 27s 272ms/step - loss: 0.6115 - accuracy: 0.8206 - val_loss: 1.5984 - val_accuracy: 0.5062
Epoch 6/30
98/98 [==============================] - 26s 268ms/step - loss: 0.5674 - accuracy: 0.8315 - val_loss: 0.9449 - val_accuracy: 0.7238
Epoch 7/30
98/98 [==============================] - 27s 273ms/step - loss: 0.5315 - accuracy: 0.8405 - val_loss: 0.6509 - val_accuracy: 0.8119

In [301]:
model.evaluate(valid_set)

33/33 [==============================] - 3s 94ms/step - loss: 0.5915 - accuracy: 0.8267


[0.5915404558181763, 0.8266756534576416]

In [299]:
np.argmax(model.predict(test_set),axis=1).shape 

34/34 [==============================] - 3s 96ms/step


(1067, 47)

In [ ]:
def generate_chorale(model, seed_chords, length):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))
    arpegio = tf.reshape(arpegio, [1, -1])
    for chord in range(length): #length表示要预测的和声数
        for note in range(4): #每个和声前后预测四次
            next_note = np.argmax(model.predict(arpegio), axis=-1)[:1, -1:]
            arpegio = tf.concat([arpegio, next_note], axis=1)
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)#置换回原来的音高数字
    return tf.reshape(arpegio, shape=[-1, 4])#置换回原来的四音和弦

Instead of always picking the note with the highest score, we will pick the next note randomly, according to the predicted probabilities. For example, if the model predicts a C3 with 75% probability, and a G3 with a 25% probability, then we will pick one of these two notes randomly, with these probabilities. We will also add a temperature parameter that will control how "hot" (i.e., daring) we want the system to feel. 
A high temperature will bring the predicted probabilities closer together, reducing the probability of the likely notes and increasing the probability of the unlikely ones.

In [336]:
def generate_chorale_v2(model, seed_chords, length, temperature=1):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))
    arpegio = tf.reshape(arpegio, [1, -1])
    for chord in range(length):
        for note in range(4):
            next_note_probas = model.predict(arpegio)[0, -1:]
            rescaled_logits = tf.math.log(next_note_probas) / temperature
            #temp参数越高，概率分布就越平均，否则越分离
            next_note = tf.random.categorical(rescaled_logits, num_samples=1)
            #从调整过权重的概率矩阵里随机挑一个
            arpegio = tf.concat([arpegio, next_note], axis=1)
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)
    return tf.reshape(arpegio, shape=[-1, 4])

In [342]:
rand_id=np.random.randint(0,len(test_chorales_list))
rand_chorale=test_chorales[rand_id]#随机取一个文件，音高转换为频率

rand_piece=np.random.randint(0,len(rand_chorale))
seed_chords=rand_chorale[0:rand_piece]
play_chords(generate_chorale_v2(model,seed_chords=seed_chords,length=rand_piece,temperature=1.2))
#整个音频后半段都是预测值

1/1 [==============================] - 0s 31ms/step
